<center><img src='https://github.com/andreivannuabc/loss_function_cyclegan/blob/main/cycle_consistenc.png?raw=true' height=350></center>
<p>
   

<h1><center> Perception Loss Function using CycleGan in Pathologic Myopia</center></h1>
<h2><center> Introduction to a Novel Loss Function </center></h2>

<h1>Package</h1>

In [1]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow_addons as tfa
from tensorflow.keras import Model, losses, optimizers

E0000 00:00:1740436034.069944      12 common_lib.cc:815] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`=local.
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:531
/usr/local/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
HEIGHT = 768
WIDTH = 768 
CHANNELS = 3
EPOCHS = 200
BATCH_SIZE = 32

In [3]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f'Running on TPU {tpu.master()}')
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()


REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')
AUTO = tf.data.experimental.AUTOTUNE
AUTO = tf.data.AUTOTUNE

Running on TPU 
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local


2025-02-24 22:27:25.285692: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2025-02-24 22:27:25.285836: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2025-02-24 22:27:25.285913: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2025-02-24 22:27:25.285986: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2025-02-24 22:27:25.286056: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2025-02-24 22:27:25.286254: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2025-02-24 22:27:25.286354: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority un

INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:localhost/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


REPLICAS: 8


In [4]:
import os
import pandas as pd
import tensorflow as tf
from PIL import Image

def get_filenames_from_folder(folder_path, extensions, limit=None):
    filenames = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(extensions):
                file_path = os.path.join(root, file)
                # Verificación opcional para asegurarse de que la imagen no esté corrupta
                try:
                    img = Image.open(file_path)
                    img.verify()  # Verifica si la imagen está corrupta
                    filenames.append(file_path)
                    if limit and len(filenames) >= limit:
                        return filenames
                except (IOError, SyntaxError) as e:
                    print(f"Imagen corrupta detectada: {file_path}")
                    continue  # Ignora la imagen corrupta
    return filenames

# Aplica la función para obtener los nombres de archivo de la Clase 1
class_1_filenamesaptos = get_filenames_from_folder("/kaggle/input/stargan-retinas/stargan_retina/normal_fundus", ('.jpg', '.jpeg', '.png', '.tif'), 5000)
class_1_filenamesodir = get_filenames_from_folder("/kaggle/input/dataset-normal-1024/normal", ('.jpg', '.jpeg', '.png', '.tif'),6000)
class_1_filenames = class_1_filenamesodir 
#class_1_filenames_train = get_filenames_from_folder("/kaggle/input/indian-diabetic-retinopathy-image-dataset/A.%20Segmentation/A. Segmentation/1. Original Images/a. Training Set", ('.jpg', '.jpeg', '.png', '.tif'), 81)
#class_1_filenames_test = get_filenames_from_folder("/kaggle/input/indian-diabetic-retinopathy-image-dataset/A.%20Segmentation/A. Segmentation/1. Original Images/b. Testing Set", ('.jpg', '.jpeg', '.png', '.tif'), 81)
class_2_filenamesa = get_filenames_from_folder("/kaggle/input/dataset-nohor-toxoplasmosis/todocropaug", ('.jpg', '.jpeg', '.png', '.tif'), 9600)
class_2_filenamesb = get_filenames_from_folder("/kaggle/input/dataset-toxoplasmosis-hor/todocropaughor", ('.jpg', '.jpeg', '.png', '.tif'), 9600)


# Función para contar los elementos
def count_data_items(filenames):
    return len(filenames)

# Combinar las listas de archivos de la clase 2
class_2_filenames = class_2_filenamesa  + class_2_filenamesb
n_class_1_samples = count_data_items(class_1_filenames)
n_class_2_samples = count_data_items(class_2_filenames)
print(f'Número de archivos Clase normal fundus: {n_class_1_samples}')
print(f'Número de archivos Clase diabetic: {n_class_2_samples}')


Número de archivos Clase normal fundus: 6000
Número de archivos Clase diabetic: 7618


In [5]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def image_example(image_string):
    feature = {
        'image': _bytes_feature(image_string),
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

def write_tfrecord(image_paths, output_file):
    with tf.io.TFRecordWriter(output_file) as writer:
        for image_path in image_paths:
            try:
                with open(image_path, 'rb') as f:
                    image_string = f.read()
                if not image_string:
                    print(f"Imagen vacía: {image_path}")
                    continue
                tf_example = image_example(image_string)
                writer.write(tf_example.SerializeToString())
            except Exception as e:
                print(f"Error al procesar {image_path}: {e}")
                continue

write_tfrecord(class_1_filenames, 'class_1.tfrecord')
write_tfrecord(class_2_filenames, 'class_2.tfrecord')


In [6]:
def decode_image(image):
    image = tf.image.decode_png(image, channels=CHANNELS)
    image = tf.image.resize(image, [HEIGHT, WIDTH])  # Redimensionar la imagen
    image = (tf.cast(image, tf.float32) / 127.5) - 1
    return image


def load_image(filename):
    image = tf.io.read_file(filename)
    image = decode_image(image)
    return image


def parse_tfrecord(example_proto):
    image_feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
    }
    parsed_features = tf.io.parse_single_example(example_proto, image_feature_description)
    image = tf.io.decode_image(parsed_features['image'], channels=CHANNELS, expand_animations=False)
    image.set_shape([None, None, CHANNELS]) 
    image = tf.image.resize(image, [HEIGHT, WIDTH])
    image = (tf.cast(image, tf.float32) / 127.5) - 1
    return image


def load_dataset(tfrecord_file):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_file)
    dataset = raw_dataset.map(parse_tfrecord, num_parallel_calls=AUTO)
    return dataset


def display_samples(ds, row, col):
    ds_iter = iter(ds)
    plt.figure(figsize=(15, int(15*row/col)))
    for j in range(row*col):
        example_batch = next(ds_iter)
        example_sample = example_batch 
        plt.subplot(row, col, j+1)
        plt.axis('off')
        plt.imshow((example_sample.numpy() * 0.5 + 0.5))
    plt.show()

    
def get_gan_dataset(monet_tfrecord, photo_tfrecord, batch_size=1):
    monet_ds = load_dataset(monet_tfrecord)
    photo_ds = load_dataset(photo_tfrecord)

    monet_ds = monet_ds.cache().shuffle(1024).repeat()
    photo_ds = photo_ds.cache().shuffle(1024).repeat()

    monet_ds = monet_ds.batch(batch_size, drop_remainder=True)
    photo_ds = photo_ds.batch(batch_size, drop_remainder=True)

    monet_ds = monet_ds.prefetch(AUTO)
    photo_ds = photo_ds.prefetch(AUTO)

    gan_ds = tf.data.Dataset.zip((monet_ds, photo_ds))
    return gan_ds



def display_generated_samples(ds, model, n_samples):
    ds_iter = iter(ds)
    for n_sample in range(n_samples):
        example_sample = next(ds_iter)
        generated_sample = model.predict(example_sample)
        
        plt.subplot(121)
        plt.title("input image")
        plt.imshow(example_sample[0] * 0.5 + 0.5)
        plt.axis('off')
        
        plt.subplot(122)
        plt.title("Generated image")
        plt.imshow(generated_sample[0] * 0.5 + 0.5)
        plt.axis('off')
        plt.show()
        
def predict_and_save(input_ds, model, output_path, domain_labels):
    i = 1
    for img, label in input_ds:
        prediction = model([img, label], training=False)[0].numpy()
        prediction = (prediction * 127.5 + 127.5).astype(np.uint8)
        im = PIL.Image.fromarray(prediction)
        im.save(f'{output_path}{str(i)}.jpg')
        i += 1


def downsample(filters, size, apply_instancenorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(L.Conv2D(filters, size, strides=2, padding='same',
                        kernel_initializer=initializer, use_bias=False))
    if apply_instancenorm:
        result.add(tfa.layers.InstanceNormalization())
    result.add(L.LeakyReLU())
    return result

def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = tf.keras.Sequential()
    result.add(L.Conv2DTranspose(filters, size, strides=2,
                                 padding='same',
                                 kernel_initializer=initializer,
                                 use_bias=False))
    result.add(tfa.layers.InstanceNormalization())
    result.add(L.ReLU())
    if apply_dropout:
        result.add(L.Dropout(0.5))
    return result

In [7]:
OUTPUT_CHANNELS = 3
# Asumiendo que HEIGHT, WIDTH, y CHANNELS están definidos
# Ejemplo: HEIGHT = 256, WIDTH = 256, CHANNELS = 3


def generator_fn():
    inputs = L.Input(shape=[HEIGHT, WIDTH, CHANNELS])

    down_stack = [
        downsample(64, 4, apply_instancenorm=False),
        downsample(128, 4),
        downsample(256, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
        downsample(512, 4),
    ]

    up_stack = [
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4),
        upsample(512, 4),
        upsample(256, 4),
        upsample(128, 4),
        upsample(64, 4),
    ]

    initializer = tf.random_normal_initializer(0., 0.02)
    last = L.Conv2DTranspose(OUTPUT_CHANNELS, 4, strides=2, padding='same',
                             kernel_initializer=initializer, activation='tanh')

    x = inputs
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)
    skips = list(reversed(skips[:-1]))

    for up, skip in zip(up_stack, skips):
        x = up(x)
        if x.shape[1] != skip.shape[1] or x.shape[2] != skip.shape[2]:
            x = tf.image.resize(x, (skip.shape[1], skip.shape[2]), method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
        x = L.Concatenate()([x, skip])

    x = last(x)
    return tf.keras.Model(inputs=inputs, outputs=x)

def discriminator_fn():
    initializer = tf.random_normal_initializer(0., 0.02)
    gamma_init = tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.02)

    inp = L.Input(shape=[HEIGHT, WIDTH, CHANNELS], name='input_image')

    x = inp
    down1 = downsample(64, 4, False)(x)
    down2 = downsample(128, 4)(down1)
    down3 = downsample(256, 4)(down2)
    down4 = downsample(512, 4)(down3)

    zero_pad1 = L.ZeroPadding2D()(down4)
    conv = L.Conv2D(512, 3, strides=1, kernel_initializer=initializer, use_bias=False)(zero_pad1)
    norm1 = tfa.layers.InstanceNormalization(gamma_initializer=gamma_init)(conv)
    leaky_relu = L.LeakyReLU()(norm1)
    zero_pad2 = L.ZeroPadding2D()(leaky_relu)
    last = L.Conv2D(1, 3, strides=1, kernel_initializer=initializer)(zero_pad2)

    return Model(inputs=inp, outputs=last)


In [8]:
with strategy.scope():
    def discriminator_loss(real, generated):
        real_loss = losses.BinaryCrossentropy(from_logits=True, reduction=losses.Reduction.NONE)(tf.ones_like(real), real)
        generated_loss = losses.BinaryCrossentropy(from_logits=True, reduction=losses.Reduction.NONE)(tf.zeros_like(generated), generated)
        total_disc_loss = real_loss + generated_loss
        return total_disc_loss * 0.5

    def generator_loss(generated):
        return losses.BinaryCrossentropy(from_logits=True, reduction=losses.Reduction.NONE)(tf.ones_like(generated), generated)
    
    with strategy.scope():
        def calc_cycle_loss(real_image, cycled_image, LAMBDA):
            loss1 = tf.reduce_mean(tf.abs(real_image - cycled_image))
            return LAMBDA * loss1

    with strategy.scope():
        def identity_loss(real_image, same_image, LAMBDA):
            loss = tf.reduce_mean(tf.abs(real_image - same_image))
            return LAMBDA * 0.5 * loss




In [9]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras import layers

def model_build(image_size, num_classes):
    inputs = tf.keras.Input(shape=(image_size, image_size, 3))
    base_model = VGG19(weights='imagenet', include_top=False, input_tensor=inputs)
    base_model.trainable = False
    
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

cnn_model = model_build(224, 2)
cnn_model.load_weights('/kaggle/input/modelo-extraccion-macular/model_macular_extractlayer.h5')
cnn_model.trainable = False

80134624/80134624 [==============================] - 2s 0us/step


In [10]:
cnn_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [11]:
layers_dict = {'0': 'conv1_1',
               '5': 'conv2_1', 
               '10': 'conv3_1', 
               '19': 'conv4_1',
               '21': 'conv4_2', 
               '28': 'conv5_1'}

style_weights = {'conv1_1': 1.0,
                 'conv2_1': 0.75,
                 'conv3_1': 0.2,
                 'conv4_1': 0.2,
                 'conv5_1': 0.2}


In [12]:
layers_dict = {
    'block2_conv1': 'conv2_1',
    'block3_conv1': 'conv2_1',

}
style_weights = {
    'conv1_2': 1.0,
    'conv2_1': 0.75,
    'conv3_1': 0.2,
    'conv4_1': 0.2,
    'conv4_2': 0.2,  # Agregar esta línea si quieres usar conv4_2
    'conv5_1': 0.2
}

def get_feature_extractor(cnn_model, layer_dict):
    outputs = []
    for layer_name in layer_dict.keys():
        layer = cnn_model.get_layer(name=layer_name)
        outputs.append(layer.output)
    model = tf.keras.Model(inputs=cnn_model.input, outputs=outputs)
    model.trainable = False
    return model

feature_extractor = get_feature_extractor(cnn_model, layers_dict)

In [13]:
def hybrid_perceptual_loss(
    real_images,
    generated_images,
    feature_extractor,  
    layers_dict,        
    style_weights,
    threshold=0.5    
):
   

    
    real_images     = (real_images + 1.0)/2.0
    generated_images= (generated_images + 1.0)/2.0

  
    mean = tf.constant([0.485, 0.456, 0.406], shape=(1,1,1,3), dtype=tf.float32)
    std  = tf.constant([0.229, 0.224, 0.225], shape=(1,1,1,3), dtype=tf.float32)

    real_images     = (real_images - mean)/std
    generated_images= (generated_images - mean)/std

  
    real_images     = tf.image.resize(real_images, (224,224))
    generated_images= tf.image.resize(generated_images, (224,224))

   
    real_features      = feature_extractor(real_images, training=False)
    generated_features = feature_extractor(generated_images, training=False)

    feature_names = list(layers_dict.values())
    total_loss = 0.0

    for real_f, gen_f, name in zip(real_features, generated_features, feature_names):
  
        shape = tf.shape(real_f)
        C = tf.cast(shape[3], tf.float32)
        H = tf.cast(shape[1], tf.float32)
        W = tf.cast(shape[2], tf.float32)

      
        diff_sq = (gen_f - real_f)**2

      
        mask = tf.cast(real_f > threshold, tf.float32)  

       
        term_pathology = - diff_sq * mask
        term_sana      = diff_sq * (1.0 - mask)
        layer_loss = tf.reduce_mean(term_pathology + term_sana)

       
        layer_loss = layer_loss / (C * H * W)

      
        lambda_l = style_weights[name]
        layer_loss *= lambda_l

        total_loss += layer_loss

    return total_loss


log_dir = './logs/gradient_logs'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
file_writer = tf.summary.create_file_writer(log_dir)

In [14]:
with strategy.scope():
    monet_generator = generator_fn() 
    photo_generator = generator_fn()

    monet_discriminator = discriminator_fn()
    photo_discriminator = discriminator_fn() 



class CycleGan(Model):
    def __init__(
        self,
        monet_generator,
        photo_generator,
        monet_discriminator,
        photo_discriminator,
        lambda_cycle=10,
    ):
        super(CycleGan, self).__init__()
        self.m_gen = monet_generator
        self.p_gen = photo_generator
        self.m_disc = monet_discriminator
        self.p_disc = photo_discriminator
        self.lambda_cycle = lambda_cycle
        self.current_epoch = 0
        self.global_step = tf.Variable(0, trainable=False, dtype=tf.int64)
        self.summary_writer = file_writer

    def compile(
        self,
        m_gen_optimizer,
        p_gen_optimizer,
        m_disc_optimizer,
        p_disc_optimizer,
        gen_loss_fn,
        disc_loss_fn,
        cycle_loss_fn,
        identity_loss_fn,
        perceptual_loss_fn=None,
        layers_dict=None,
        style_weights=None,
        perceptual_lambda=0.1
    ):
        super(CycleGan, self).compile()
        self.m_gen_optimizer = m_gen_optimizer
        self.p_gen_optimizer = p_gen_optimizer
        self.m_disc_optimizer = m_disc_optimizer
        self.p_disc_optimizer = p_disc_optimizer
        self.gen_loss_fn = gen_loss_fn
        self.disc_loss_fn = disc_loss_fn
        self.cycle_loss_fn = cycle_loss_fn
        self.identity_loss_fn = identity_loss_fn
        self.perceptual_loss_fn = perceptual_loss_fn
        self.layers_dict = layers_dict
        self.style_weights = style_weights
        self.perceptual_lambda = perceptual_lambda

    def train_step(self, batch_data):
        real_monet, real_photo = batch_data
    
        with tf.GradientTape(persistent=True) as tape:
            fake_monet = self.m_gen(real_photo, training=True)
            cycled_photo = self.p_gen(fake_monet, training=True)
            fake_photo = self.p_gen(real_monet, training=True)
            cycled_monet = self.m_gen(fake_photo, training=True)
    
            same_monet = self.m_gen(real_monet, training=True)
            same_photo = self.p_gen(real_photo, training=True)
    
            disc_real_monet = self.m_disc(real_monet, training=True)
            disc_real_photo = self.p_disc(real_photo, training=True)
            disc_fake_monet = self.m_disc(fake_monet, training=True)
            disc_fake_photo = self.p_disc(fake_photo, training=True)
    
            monet_gen_loss = self.gen_loss_fn(disc_fake_monet)
            photo_gen_loss = self.gen_loss_fn(disc_fake_photo)
            total_cycle_loss = self.cycle_loss_fn(real_monet, cycled_monet, self.lambda_cycle) + \
                               self.cycle_loss_fn(real_photo, cycled_photo, self.lambda_cycle)
            monet_id_loss = self.identity_loss_fn(real_monet, same_monet, self.lambda_cycle)
            photo_id_loss = self.identity_loss_fn(real_photo, same_photo, self.lambda_cycle)
    
            total_monet_gen_loss = monet_gen_loss + total_cycle_loss + monet_id_loss
            total_photo_gen_loss = photo_gen_loss + total_cycle_loss + photo_id_loss
            """
            if self.perceptual_loss_fn is not None:
                perc_loss_monet = self.perceptual_loss_fn(
                    real_monet, 
                    fake_monet, 
                    feature_extractor, 
                    self.layers_dict, 
                    self.style_weights,
                     threshold=0.5 
                )
                perc_loss_photo = self.perceptual_loss_fn(
                    real_photo, 
                    fake_photo, 
                    feature_extractor, 
                    self.layers_dict, 
                    self.style_weights,
                     threshold=0.5 
                )
                total_monet_gen_loss += self.perceptual_lambda * perc_loss_monet
                total_photo_gen_loss += self.perceptual_lambda * perc_loss_photo
               """
            monet_disc_loss = self.disc_loss_fn(disc_real_monet, disc_fake_monet)
            photo_disc_loss = self.disc_loss_fn(disc_real_photo, disc_fake_photo)
    
        monet_generator_gradients = tape.gradient(total_monet_gen_loss, self.m_gen.trainable_variables)
        photo_generator_gradients = tape.gradient(total_photo_gen_loss, self.p_gen.trainable_variables)
        monet_discriminator_gradients = tape.gradient(monet_disc_loss, self.m_disc.trainable_variables)
        photo_discriminator_gradients = tape.gradient(photo_disc_loss, self.p_disc.trainable_variables)
    
        self.m_gen_optimizer.apply_gradients(zip(monet_generator_gradients, self.m_gen.trainable_variables))
        self.p_gen_optimizer.apply_gradients(zip(photo_generator_gradients, self.p_gen.trainable_variables))
        self.m_disc_optimizer.apply_gradients(zip(monet_discriminator_gradients, self.m_disc.trainable_variables))
        self.p_disc_optimizer.apply_gradients(zip(photo_discriminator_gradients, self.p_disc.trainable_variables))
    
        self.global_step.assign_add(1)
    
        # Asegúrate que las métricas retornadas sean escalares (usamos tf.reduce_mean)
        return {
            'monet_gen_loss': tf.reduce_mean(total_monet_gen_loss),
            'photo_gen_loss': tf.reduce_mean(total_photo_gen_loss),
            'monet_disc_loss': tf.reduce_mean(monet_disc_loss),
            'photo_disc_loss': tf.reduce_mean(photo_disc_loss)
        }

2025-02-24 22:30:54.227364: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2025-02-24 22:31:05.609717: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.


In [15]:

class LearningRateLoggerCallback(tf.keras.callbacks.Callback):
    def __init__(self, csv_file='lr_log.csv', monitor='monet_gen_loss'):
        super().__init__()
        self.csv_file = csv_file
        self.monitor = monitor
        self.prev_lrs = None
        self.file_initialized = False

    def on_train_begin(self, logs=None):
     
        with open(self.csv_file, 'w') as f:
            f.write("epoch,"+self.monitor+",m_gen_lr,p_gen_lr,m_disc_lr,p_disc_lr\n")

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        metric_value = logs.get(self.monitor, None)

      
        m_gen_lr = float(tf.keras.backend.get_value(self.model.m_gen_optimizer.lr))
        p_gen_lr = float(tf.keras.backend.get_value(self.model.p_gen_optimizer.lr))
        m_disc_lr = float(tf.keras.backend.get_value(self.model.m_disc_optimizer.lr))
        p_disc_lr = float(tf.keras.backend.get_value(self.model.p_disc_optimizer.lr))

   
        with open(self.csv_file, 'a') as f:
            f.write(f"{epoch},{metric_value},{m_gen_lr},{p_gen_lr},{m_disc_lr},{p_disc_lr}\n")

       
        current_lrs = (m_gen_lr, p_gen_lr, m_disc_lr, p_disc_lr)
        if self.prev_lrs is not None:
            
            if current_lrs != self.prev_lrs:
                print(f"Epoch {epoch}: ¡Los learning rates han cambiado!")
                print(f"Antes: {self.prev_lrs}, Ahora: {current_lrs}")
        self.prev_lrs = current_lrs


In [16]:
def load_image(filepath, new_size=(224,224)):

    img_bytes = tf.io.read_file(filepath)
    img = tf.image.decode_jpeg(img_bytes, channels=3)
    img = tf.image.resize(img, new_size)
    img = tf.cast(img, tf.float32) / 127.5 - 1.0 
    img = tf.expand_dims(img, axis=0)  
    return img


In [17]:

class DisplaySampleCallback(tf.keras.callbacks.Callback):
    def __init__(self, image_path, monet_generator, output_dir='./sample_epochs', freq=1):
        super().__init__()
        self.image_path = image_path
        self.monet_generator = monet_generator
        self.freq = freq
        self.output_dir = output_dir

        
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)

        
        self.sample_photo = self._load_and_preprocess_image(self.image_path)

    def _load_and_preprocess_image(self, image_path):
     
        img = Image.open(image_path).convert('RGB')  
        img = img.resize((768, 768)) 
        img_array = np.array(img).astype('float32') / 127.5 - 1.0  
        return tf.expand_dims(img_array, axis=0)  

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.freq == 0:
          
            generated_monet = self.monet_generator(self.sample_photo, training=False)
            gen_img = (generated_monet[0].numpy() * 0.5) + 0.5 

     
            plt.figure(figsize=(6, 6))
            plt.imshow(gen_img)
            plt.axis('off')
            plt.title(f"Epoch {epoch + 1}")
            plt.savefig(os.path.join(self.output_dir, f"epoch_{epoch + 1}.png"))
            plt.close()


class CustomTrainCheckpoints(tf.keras.callbacks.Callback):
  
    def __init__(self, checkpoint_dir='checkpoints', save_freq=5):
       
        super().__init__()
        self.checkpoint_dir = checkpoint_dir
        self.save_freq = save_freq
        if not os.path.exists(self.checkpoint_dir):
            os.makedirs(self.checkpoint_dir)

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
         
            ckpt = tf.train.Checkpoint(
                monet_generator=self.model.m_gen,
                photo_generator=self.model.p_gen,
                monet_discriminator=self.model.m_disc,
                photo_discriminator=self.model.p_disc,
                monet_gen_optimizer=self.model.m_gen_optimizer,
                photo_gen_optimizer=self.model.p_gen_optimizer,
                monet_disc_optimizer=self.model.m_disc_optimizer,
                photo_disc_optimizer=self.model.p_disc_optimizer
            )
            ckpt_path = os.path.join(self.checkpoint_dir, f'ckpt_epoch_{epoch+1}')
            ckpt.save(file_prefix=ckpt_path)
            print(f"Checkpoint guardado: {ckpt_path}")


In [18]:
gan_ds = get_gan_dataset('class_2.tfrecord', 'class_1.tfrecord', batch_size=BATCH_SIZE)
steps_per_epoch = n_class_2_samples // BATCH_SIZE

In [19]:

log_dir = './logs/gradient_logs'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
file_writer = tf.summary.create_file_writer(log_dir)

In [20]:
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [ ]:

from tensorflow.keras.callbacks import ReduceLROnPlateau

with strategy.scope():
  
    monet_generator = generator_fn()
    photo_generator = generator_fn()
    monet_discriminator = discriminator_fn()
    photo_discriminator = discriminator_fn()

  
    monet_generator_optimizer = optimizers.Adam(2e-4, beta_1=0.5)
    photo_generator_optimizer = optimizers.Adam(2e-4, beta_1=0.5)
    monet_discriminator_optimizer = optimizers.Adam(2e-4, beta_1=0.5)
    photo_discriminator_optimizer = optimizers.Adam(2e-4, beta_1=0.5)


    gan_model = CycleGan(
        monet_generator, 
        photo_generator,
        monet_discriminator, 
        photo_discriminator,
        lambda_cycle=10
    )
  
    gan_model.compile(
        m_gen_optimizer=monet_generator_optimizer,
        p_gen_optimizer=photo_generator_optimizer,
        m_disc_optimizer=monet_discriminator_optimizer,
        p_disc_optimizer=photo_discriminator_optimizer,
        gen_loss_fn=generator_loss,
        disc_loss_fn=discriminator_loss,
        cycle_loss_fn=calc_cycle_loss,
        identity_loss_fn=identity_loss,
        perceptual_loss_fn=hybrid_perceptual_loss,
        layers_dict=layers_dict,
        style_weights=style_weights,
        perceptual_lambda=0.1
    )

    # 4. Callbacks
    display_callback = DisplaySampleCallback(
        image_path='/kaggle/input/dataset-normal-1024/normal/002c2135.jpg',
        monet_generator=gan_model.m_gen,     
        output_dir='./sample_epochs',    
        freq=1                              
    )
    reduce_lr_callback = ReduceLROnPlateau(
        monitor='monet_gen_loss',
        factor=0.5,
        patience=5,
        verbose=1,
        mode='min'
    )

    checkpoint_callback = CustomTrainCheckpoints(
        checkpoint_dir='checkpoints',
        save_freq=25
    )


    history = gan_model.fit(
        gan_ds,
        steps_per_epoch=steps_per_epoch,
        epochs=200,
        verbose=2,
        callbacks=[display_callback, reduce_lr_callback, checkpoint_callback]
    )

Epoch 1/200


2025-02-24 22:31:13.430933: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2025-02-24 22:31:13.448253: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2025-02-24 22:31:13.465966: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2025-02-24 22:31:13.482462: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2025-02-24 22:31:13.499260: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2025-02-24 22:31:13.516345: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority unimplemented for this stream.
2025-02-24 22:31:13.533931: E ./tensorflow/compiler/xla/stream_executor/stream_executor_internal.h:124] SetPriority un

238/238 - 283s - monet_gen_loss: 1.9354 - photo_gen_loss: 1.9564 - monet_disc_loss: 0.7242 - photo_disc_loss: 0.4723 - lr: 0.0010 - 283s/epoch - 1s/step
Epoch 2/200
238/238 - 177s - monet_gen_loss: 2.0131 - photo_gen_loss: 1.5677 - monet_disc_loss: 0.7205 - photo_disc_loss: 0.7491 - lr: 0.0010 - 177s/epoch - 745ms/step
Epoch 3/200
238/238 - 177s - monet_gen_loss: 1.7217 - photo_gen_loss: 1.6029 - monet_disc_loss: 0.6363 - photo_disc_loss: 0.6281 - lr: 0.0010 - 177s/epoch - 746ms/step
Epoch 4/200
238/238 - 177s - monet_gen_loss: 1.5900 - photo_gen_loss: 2.1965 - monet_disc_loss: 0.4816 - photo_disc_loss: 0.5178 - lr: 0.0010 - 177s/epoch - 745ms/step
Epoch 5/200
238/238 - 177s - monet_gen_loss: 1.6350 - photo_gen_loss: 1.6893 - monet_disc_loss: 0.6036 - photo_disc_loss: 0.6894 - lr: 0.0010 - 177s/epoch - 745ms/step
Epoch 6/200
238/238 - 177s - monet_gen_loss: 1.9049 - photo_gen_loss: 2.0192 - monet_disc_loss: 0.5910 - photo_disc_loss: 0.5724 - lr: 0.0010 - 177s/epoch - 745ms/step
Epoch 7

In [ ]:
# =====================================
# Celda 3: REANUDAR ENTRENAMIENTO
# =====================================

# Volvemos a usar las mismas definiciones de generator_fn(), discriminator_fn(), etc.

with strategy.scope():
    # 1. Crear redes con la misma arquitectura
    monet_generator_resumed = generator_fn()
    photo_generator_resumed = generator_fn()
    monet_discriminator_resumed = discriminator_fn()
    photo_discriminator_resumed = discriminator_fn()

    # 2. Crear los mismos optimizadores
    monet_generator_optimizer_resumed = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    photo_generator_optimizer_resumed = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    monet_discriminator_optimizer_resumed = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
    photo_discriminator_optimizer_resumed = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

    # 3. Instanciar un nuevo CycleGAN
    gan_model_resumed = CycleGan(
        monet_generator_resumed,
        photo_generator_resumed,
        monet_discriminator_resumed,
        photo_discriminator_resumed,
        lambda_cycle=10
    )
    gan_model_resumed.compile(
        m_gen_optimizer=monet_generator_optimizer_resumed,
        p_gen_optimizer=photo_generator_optimizer_resumed,
        m_disc_optimizer=monet_discriminator_optimizer_resumed,
        p_disc_optimizer=photo_discriminator_optimizer_resumed,
        gen_loss_fn=generator_loss,
        disc_loss_fn=discriminator_loss,
        cycle_loss_fn=calc_cycle_loss,
        identity_loss_fn=identity_loss,
        perceptual_loss_fn=perceptual_loss,
        layers_dict=layers_dict,
        style_weights=style_weights,
        perceptual_lambda=0.1
    )

    # 4. Crear un tf.train.Checkpoint para restaurar estado (G, D y Opts)
    ckpt_resumed = tf.train.Checkpoint(
        monet_generator=gan_model_resumed.m_gen,
        photo_generator=gan_model_resumed.p_gen,
        monet_discriminator=gan_model_resumed.m_disc,
        photo_discriminator=gan_model_resumed.p_disc,
        monet_gen_optimizer=gan_model_resumed.m_gen_optimizer,
        photo_gen_optimizer=gan_model_resumed.p_gen_optimizer,
        monet_disc_optimizer=gan_model_resumed.m_disc_optimizer,
        photo_disc_optimizer=gan_model_resumed.p_disc_optimizer
    )

    # 5. Indica el checkpoint a cargar (ej. el de la época 30)
    #    asumiendo que lo guardó como: checkpoints/ckpt_epoch_30-1
    #    (o similar, usa tf.train.latest_checkpoint para encontrarlo)
    checkpoint_dir = '/kaggle/input/checkpoint30-diffusse-myopia' 
    latest_ckpt = tf.train.latest_checkpoint(checkpoint_dir)
    if latest_ckpt:
        ckpt_resumed.restore(latest_ckpt).expect_partial()
        print(f"Checkpoint restaurado desde {latest_ckpt}")
    else:
        print("No se encontró checkpoint en la carpeta:", checkpoint_dir)

    # 6. Callbacks para seguir entrenando
    display_callback = DisplaySampleCallback(
        image_path='/kaggle/input/dataset-normal-1024/normal/002c2135.jpg',  # Replace with your image path
        monet_generator=gan_model.m_gen,     # Your Monet generator model
        output_dir='./sample_epochs',        # Directory to save generated images
        freq=1                               # Frequency of saving images (every epoch)
    )
    reduce_lr_callback_resumed = ReduceLROnPlateau(
        monitor='monet_gen_loss',
        factor=0.5,
        patience=5,
        verbose=1,
        mode='min'
    )
    checkpoint_callback_resumed = CustomTrainCheckpoints(
        checkpoint_dir='checkpoints_resumed',  # carpeta distinta o la misma
        save_freq=25
    )

    # 7. Reanudar entrenamiento: si lo dejaste en la 30, pon initial_epoch=30
    #    para que la siguiente sea la 31
    history_resumed = gan_model_resumed.fit(
        gan_ds,
        steps_per_epoch=steps_per_epoch,
        epochs=60,         # entrenar otras 30 epocas, por ejemplo
        initial_epoch=30,  # arranca contaje en 30
        verbose=2,
        callbacks=[display_callback_resumed, reduce_lr_callback_resumed, checkpoint_callback_resumed]
    )


In [ ]:

p_gen_checkpoint = '/kaggle/input/modelos-epoca38-myopia/gan_model_myopiaNoAttention_epoch_38.h5'
m_gen_checkpoint = '/kaggle/working/model_checkpoints/gan_model_myopiaNoAttention_epoch_38.h5'

print(f"¿Existe {p_gen_checkpoint}? {os.path.exists(p_gen_checkpoint)}")
print(f"¿Existe {m_gen_checkpoint}? {os.path.exists(m_gen_checkpoint)}")


In [ ]:
gan_model.p_gen.save('gan_model_normal_myopiaNoAttention66.h5')  # Guarda el generador que transforma fotos en Monet

In [ ]:
gan_model.m_gen.save('gan_model_myopiaNoAttention66.h5')  # Guarda el generador que transforma fotos en Monet

In [ ]:

import zipfile

def folder_to_zip(folder_path, output_zip_path):
    # Crear un archivo .zip
    with zipfile.ZipFile(output_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Recorrer todos los archivos en el directorio
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                # Crear la ruta completa del archivo
                full_path = os.path.join(root, file)
                # Agregar el archivo al zip, eliminando el prefijo de la carpeta original
                zipf.write(full_path, os.path.relpath(full_path, folder_path))

    print(f"Carpeta {folder_path} comprimida en {output_zip_path}")


In [ ]:
folder_to_zip('/kaggle/working/checkpoints', 'checkpointsnocnnpatchy100.zip')

In [ ]:
folder_to_zip('/kaggle/working/sample_epochs', 'sample_epochs_resumednocnnpatchy100.zip')

In [ ]:
history.keys()

In [ ]:
# Suponiendo que tienes un objeto de historial que contiene los datos de entrenamiento
for key in history.keys():
    data = np.array(history[key])  # Asegúrate de que 'history[key]' tiene la forma correcta
    # Verificar la forma de los datos
    print(key, data.shape)

    # Si los datos son de alta dimensión, podríamos promediarlos
    # Asegúrate de que este enfoque tiene sentido para tus datos específicos
    if data.ndim > 2:
        data = data.mean(axis=tuple(range(1, data.ndim)))

    plt.plot(data, label=key)
    

plt.title('Curvas de Pérdida durante el Entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show() 

In [ ]:

# Suponiendo que tienes un objeto de historial que contiene los datos de entrenamiento
for key in history.keys():
    data = np.array(history[key])  # Asegúrate de que 'history[key]' tiene la forma correcta
    # Verificar la forma de los datos
    print(key, data.shape)

    # Si los datos son de alta dimensión, podríamos promediarlos
    # Asegúrate de que este enfoque tiene sentido para tus datos específicos
    if data.ndim > 2:
        data = data.mean(axis=tuple(range(1, data.ndim)))

    plt.plot(data, label=key)

plt.title('Curvas de Pérdida durante el Entrenamiento')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()



In [ ]:
display_generated_samples(load_dataset('class_1.tfrecord').batch(1), monet_generator, 25)


In [ ]:
display_generated_samples(load_dataset('class_2.tfrecord').batch(1), photo_generator, 25)


In [ ]:
os.makedirs('../images/') # Create folder to save generated images

predict_and_save(load_dataset(class_normal_filenames).batch(1), monet_generator, '../images/')

In [ ]:
shutil.make_archive('/kaggle/working/images/', 'zip', '../images')

print(f"Generated samples: {len([name for name in os.listdir('../images/') if os.path.isfile(os.path.join('../images/', name))])}")

In [ ]:

# Definiciones globales
HEIGHT = 512
WIDTH = 512
CHANNELS = 3
AUTO = tf.data.experimental.AUTOTUNE

# Función para decodificar y preprocesar imágenes
def decode_image(image):
    image = tf.image.decode_png(image, channels=CHANNELS)
    image = tf.image.resize(image, [HEIGHT, WIDTH])
    image = (tf.cast(image, tf.float32) / 127.5) - 1
    return image

def load_image(filename):
    image = tf.io.read_file(filename)
    image = decode_image(image)
    return image

def load_dataset(filenames):
    dataset = tf.data.Dataset.from_tensor_slices(filenames)
    dataset = dataset.map(load_image, num_parallel_calls=AUTO)
    return dataset


import tensorflow_addons as tfa  # Importa tensorflow_addons



# Función para hacer predicciones
def predict_image(model, image_path):
    image = load_image(image_path)
    image = tf.expand_dims(image, axis=0)  # Añadir la dimensión de batch
    prediction = model.predict(image)
    prediction = (prediction + 1) * 0.5  # Transformar de [-1, 1] a [0, 1]
    return prediction
# helper function for un-normalizing an image 
# and converting it from a Tensor image to a NumPy image for display
def im_convert(tensor):
    """ Display a tensor as an image. """
    
    image = tensor.to("cpu").clone().detach()
    image = image.numpy().squeeze()
    image = image.transpose(1,2,0)
    image = image * np.array((0.229, 0.224, 0.225)) + np.array((0.485, 0.456, 0.406))
    image = image.clip(0, 1)

    return image
# Visualización de una imagen de predicción



def display_prediction(image_path):
    plt.figure(figsize=(6, 6))
    prediction = predict_image(gan_model.m_gen, image_path)
    
    # Remove batch dimension if necessary
    prediction = prediction.squeeze()  # Remove batch dimension

    # Clip values to the expected range [0, 1] if not already
    prediction = np.clip(prediction, 0, 1)

    # Convert to 8-bit per channel format for image representation
    prediction = (prediction * 255).astype(np.uint8)

    # Display using matplotlib
    plt.imshow(prediction)
    plt.axis('off')
    plt.show()

    # Save using PIL
    img = Image.fromarray(prediction)
    img.save("reconstruccion2.png")  # Saving as PNG

# Use the function with an appropriate image path
image_path = '/kaggle/input/aptos-crop/train_images_crop/0c917c372572.png'
display_prediction(image_path)


In [ ]:
image_path = '/kaggle/input/aptos-crop/train_images_crop/0e0fc1d9810c.png'
display_prediction(image_path)


In [ ]:
image_path = '/kaggle/input/aptos-crop/train_images_crop/0a4e1a29ffff.png'
display_prediction(image_path)

In [ ]:
image_path = '/kaggle/input/aptos-crop/train_images_crop/1409ab48175a.png'
display_prediction(image_path)

#### Use the function with an appropriate image path
image_path = '/kaggle/input/aptos-crop/train_images_crop/02da652c74b8.png'
display_prediction(image_path)
